In [ ]:
import random
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import sys
sys.path.append('.')   # path for local package
from plot_tools import *

In [ ]:
class simulation_single:
    def __init__(self, pop, stoichs, rates):
        self.stoichs = stoichs #stoichiometry indices
        self.rates = rates #rates for each chemical reaction
        self.pop = pop #populatio

    def propensities(self,pop, rank):
      '''Dictionary of the ranks of the chemical reaction. Assumes uni-molecular reaction
      Input:
        rank: the stoichiometry index of the reactant
        pop: the population of the reactant
      Output:
        dict[rank]: the propensity function of the reaction
      '''
      dict = {
        0: 1, 
        1: pop,
        2: pop*(pop-1),
        3: pop*(pop-1)*(pop-2),
      }
      return dict[rank]

    def props_initialize(self):
      '''Calculates the propensity (transition rate) of all the chemical reactions
      Output: array of the propensities of the chemical reactions'''
      props = []
      props_reacts = []
      props_reacts = [self.propensities(self.pop, self.stoichs[i][0])*self.rates[i] for i in range(len(self.rates))]
      props.append(props_reacts)
      return props[0]
    
    def reverse_inter(self):
      '''Identifies the reverse chemical reaction for each reaction.'''
      rev_stoich, rev_rates, num_react = self.stoichs[:], self.rates[:], len(self.stoichs)
      for i in range(num_react):
        rev_stoich[i] = [self.stoichs[i][1], self.stoichs[i][0]] #stoichiometry indices of reverse reaction
        index = None 
        for j, sub_list in enumerate(self.stoichs): #finds index of reverse reaction compared to initial array
            if sub_list == rev_stoich[i]:
                index = j
        rev_rates[i] = self.rates[index] #identifies its rate
      return rev_stoich, rev_rates

    def entropy_prod(self, prop, rev_stoich, rev_rate):
      '''Calculates the entropy production rate for given chemical reaction'''
      rev_prop = self.propensities(self.pop+(rev_stoich[0]-rev_stoich[1]), rev_stoich[0])*rev_rate  
      return prop*np.log(prop/rev_prop)

    def update_events(self):
      '''Implements the gillespie algorithm with reactions.
      It updates the time and the population.
      If entropy=='on' output is the entropy production.
      '''
      r1, r2 = random.random(), random.random()
      a0 = sum(self.props)
      dt = 1/a0*np.log(1/r1) #gillespie
      cum_sum= 0

      if self.entropy == 'on': 
          rev_stoich, rev_rates = self.reverse_inter()

      for i in range(len(self.stoichs)): #gillespie_2: checks which reaction takes place
        cum_sum += self.props[i]/a0
        if r2<cum_sum:
            if self.entropy == 'on':
              entropy_value = self.entropy_prod(self.props[i], rev_stoich[i], rev_rates[i])
            
            self.pop += self.stoichs[i][1] - self.stoichs[i][0] #difference population depends on difference of stoich. indices
            self.props = [self.propensities(self.pop, self.stoichs[i][0])*self.rates[i] for i in range(len(self.rates))] #updates the changed propensities
            break
      self.t +=dt
      if self.entropy == 'on':
        return entropy_value

    def physical_t(self,t_array,array):
      '''Used for evolve_diff to fill in the evolution of the population the timesteps where nothing happens
      For given Dt it fills t_array and array to have timestep Dt. 'Freezes' values between intervals. '''
      T = np.arange(0,self.t_final+self.Dt, self.Dt)
      A = np.zeros_like(T,dtype=float)
      
      current_val = array[0]
      change_index = 0
      for i, time in enumerate(T):
        if i>0 and time>=t_array[change_index+1]:
          change_index+=1
          current_val = array[change_index]
        A[i]=current_val
      return A, T

    def run(self,t_final,Dt,entropy):
        '''Applies Gillespie algorithm up to a final time
        Output:
          P: Array with each element is the population array at a time
          T: the timesteps until final time
          EPR: Array with each element is the entropy production. Given only if entropy=='on'. '''
        self.t, self.Dt, self.t_final =0, Dt, t_final
        self.entropy = entropy
        pop_array, entropy_array, t_array = [np.copy(self.pop)], [0], [0]
        self.props = self.props_initialize() 
        while self.t < t_final:
          if self.entropy == 'on':
            entropy_val = self.update_events()
            entropy_array.append(entropy_val)
          else:
            self.update_events()
          pop_array.append(np.copy(self.pop)) #np.copy used so that each element is a unique copy and does not change
          t_array.append(self.t)

        P, T = self.physical_t(t_array,pop_array)
        if self.entropy == 'on':
          EPR, T = self.physical_t(t_array,entropy_array)
          return P, T, EPR
        else:
          return P, T
    
def average_simulations(simulations):
    """
    Compute the average and standard deviation of multiple simulation results.
    
    Parameters:
        simulations (list of arrays): List containing population arrays from multiple simulations.
        
    Returns:
        tuple: A tuple containing the mean and standard deviation arrays.
    """
    # Stack population arrays along a new axis
    stacked_populations = np.stack(simulations, axis=0)
    
    # Calculate mean and standard deviation across simulations
    mean_population = np.mean(stacked_populations, axis=0)
    std_population = np.std(stacked_populations, axis=0)
    
    return mean_population, std_population


### Creation-Annihilation reaction

In [ ]:
stoichs= [[0,1], [1,0]]
rates = [1,0.1]
initial = 0
t_final = 1000
dt = 0.01
simulations = []
entropy_sim = []
entropy = 'on'
for _ in range(50):
    sim = simulation_single(initial, stoichs, rates)
    if entropy == 'on':
        P,T, EPR = sim.run(t_final,dt,entropy)
        simulations.append(P)
        entropy_sim.append(EPR)
    else:
        P,T = sim.run(t_final,dt,entropy)
        simulations.append(P)     
mean_population, std_population = average_simulations(simulations)
plot_with_std(mean_population, std_population, T,'blue', 'Mean Population')
plt.show()

mean_entropy, std_entropy = average_simulations(entropy_sim)
plot_with_std(mean_entropy, std_entropy, T,'red', 'Mean Entropy')

  

## Schlogl model of 2nd kind

In [ ]:
stoichs= [[0,1], [1,0], [2,3], [3,2]]
k3 = 2200/60
k4= 37.5/60
k1 = 0.18/60
k2 = 2.5*1e-4/60
rates = [k3,k4,k1,k2]

initial = 0
t_final = 6000
dt =0.01

sim = simulation_single(initial, stoichs, rates)
P, T =sim.run(t_final, dt, 'off')
plot_pop_traj(P,T)



# Reaction-Diffusion

In [ ]:
class simulation:
    def __init__(self, pop, stoichs, rates, d):
        self.stoichs = stoichs
        self.rates = rates
        self.pop = pop
        self.d = d

    def propensities(self,pop, rank):
      '''Dictionary of the ranks of the chemical reaction. Assumes uni-molecular reaction
      Input:
        rank: the stoichiometry index of the reactant
        pop: the population of the reactant
      Output:
        dict[rank]: the propensity function of the reaction
      '''
      dict = {
        0: 1, 
        1: pop,
        2: pop*(pop-1),
        3: pop*(pop-1)*(pop-2),
      }
      return dict[rank]

    def nn_index(self,shape, dim, L):
        '''Given the dimension (e.g. x, y or z axis) it gives the indices of the neighboring boxes on that direction'''
        right, left = np.array(shape), np.array(shape)
        right[dim] += 1
        left[dim] -= 1
        return tuple(right%L), tuple(left%L) #implements periodic boundary conditions

    def props_initialize(self):
      '''Calculates the propensity (transition rate) of all the chemical reactions and diffusion
      Output:
      Multi-array with:
      1st element: multi-array with as many sub-arrays as 2d (cardinality). Each sub-array has element the propensitiy of diffusion of each box.
      2nd element: matrix with elements the array of chemical propensity of each box '''
      size=self.pop.shape
      props = [] #propensities
      prop_side = np.ones(size) #propensities of diffusion for each side e.g. left, right, up, down
      props_reacts = np.empty(size, dtype=object) #propensities of reactions for each box

      for box_index in np.ndindex(self.pop.shape):
          prop_side[box_index] = self.pop[box_index]*self.d
          props_reacts[box_index] = np.array([self.propensities(self.pop[box_index], self.stoichs[i][0])*self.rates[i] for i in range(len(self.rates))])

      props.append(prop_side)
      props.append(props_reacts)
      return props
    
    def reverse_inter(self):
      '''Identifies the reverse chemical reaction for each reaction.'''
      rev_stoich, rev_rates, num_react = self.stoichs[:], self.rates[:], len(self.stoichs)
      for i in range(num_react):
        rev_stoich[i] = [self.stoichs[i][1], self.stoichs[i][0]]
        index = None
        for j, sub_list in enumerate(self.stoichs):
            if sub_list == rev_stoich[i]:
                index = j
        rev_rates[i] = self.rates[index]
      return rev_stoich, rev_rates

    def chem_entropy_prod(self, prop, pop_box, rev_stoich, rev_rate):
      '''Calculates the entropy production rate for given chemical reaction and box'''
      rev_prop = self.propensities(pop_box+(rev_stoich[0]-rev_stoich[1]), rev_stoich[0])*rev_rate  
      return prop*np.log(prop/rev_prop)

    def diff_entropy_prod(self, box_index, nn_index):
      '''Calculates the entropy production rate for given diffusion and box'''
      rev_prop = (self.pop[nn_index]+1)*self.d
      prop = (self.pop[box_index])*self.d
      return prop*np.log(prop/rev_prop)

    def update_events(self):
      '''Implements the gillespie algorithm with diffusion and reactions'''
      r1, r2 = random.random(), random.random()
      dims = self.pop.ndim #dimension of lattice
      sum_dir, sum_chem = np.sum(self.props[0]), np.sum(np.sum(self.props[1]))
      a0 =  sum_dir*dims*2 + sum_chem
      dt = 1/a0*np.log(1/r1) #gillespie
      cum_sum= 0
      cum_direction  = sum_dir/a0
      found_condition = False #flag variable when a reaction is done then quits

      if self.entropy == 'on':
          rev_stoich, rev_rates = self.reverse_inter()

      for dim_index in range(dims): #loop for diffusion for each dimension
          L_dim = self.pop.shape[dim_index] 
          if r2<(2*dim_index+1)*cum_direction: #if +1-side of dim occurs (e.g. right diffusion)
              cum_sum = 2*dim_index*cum_direction # for 1D, this is 0. For 2D it is 0 or 2cum_direction
              for box_index in np.ndindex(self.pop.shape):
                  cum_sum += self.props[0][box_index]/a0
                  if r2<cum_sum:                         
                      right,_ = self.nn_index(box_index, dim_index, L_dim) #calculates one neighbor. Assumes square lattice
                      if self.entropy == 'on':
                        self.entropy_boxes[box_index] += self.diff_entropy_prod(box_index, right)
                        
                      # updates changed population and affected propensities 
                      self.pop[box_index] -= 1
                      self.pop[right] +=1
                      self.props[0][box_index], self.props[1][box_index] = self.pop[box_index]*self.d, np.array([self.propensities(self.pop[box_index],self.stoichs[i][0])*self.rates[i] for i in range(len(self.rates))])
                      self.props[0][right], self.props[1][right] = self.pop[right]*self.d, np.array([self.propensities(self.pop[right],self.stoichs[i][0])*self.rates[i] for i in range(len(self.rates))])
                      
                      found_condition = True
                      break
              if found_condition:
                  break
          elif r2<(2*dim_index+2)*cum_direction: #checks if -1-side of dim occurs (e.g. left diffusion)
              cum_sum = (2*dim_index+1)*cum_direction
              for box_index in np.ndindex(self.pop.shape):
                  cum_sum += self.props[0][box_index]/a0
                  if r2<cum_sum:
                      _,left = self.nn_index(box_index, dim_index, L_dim)
                      if self.entropy == 'on':
                        self.entropy_boxes[box_index] += self.diff_entropy_prod(box_index, left)
                        
                      self.pop[box_index] -= 1
                      self.pop[left]+=1
                      self.props[0][box_index], self.props[1][box_index] = self.pop[box_index]*self.d, np.array([self.propensities(self.pop[box_index],self.stoichs[i][0])*self.rates[i] for i in range(len(self.rates))])
                      self.props[0][left], self.props[1][left] = self.pop[left]*self.d, np.array([self.propensities(self.pop[left],self.stoichs[i][0])*self.rates[i] for i in range(len(self.rates))])
                      
                      found_condition = True
                      break
              if found_condition: #if diffusion occurs, stop checking other dimensions
                break
      if not found_condition: #if diffusion has not occured, check chemical reactions
        cum_sum = (2*dims)*cum_direction #diffusion on ALL sides did not occur => add propensities
        for box_index in np.ndindex(self.pop.shape):
            for react_index in range(len(self.rates)): #for each chemical reaction
                cum_sum += self.props[1][box_index][react_index]/a0
                if r2<cum_sum:
                    if self.entropy == 'on':
                       self.entropy_boxes[box_index] += self.chem_entropy_prod(self.props[1][box_index][react_index], self.pop[box_index], rev_stoich[react_index], rev_rates[react_index])

                    self.pop[box_index] += self.stoichs[react_index][1] - self.stoichs[react_index][0]
                    self.props[0][box_index], self.props[1][box_index] = self.pop[box_index]*self.d, np.array([self.propensities(self.pop[box_index],self.stoichs[i][0])*self.rates[i] for i in range(len(self.rates))])
                    found_condition = True
                    break
            if found_condition:
                break
      self.t +=dt
      if self.entropy == 'on':
        return self.entropy_boxes

    def physical_t(self,t_array,array):
      '''Used for evolve_diff to fill in the evolution of the population the timesteps where nothing happens
      For given Dt it fills t_array and array to have timestep Dt. 'Freezes' values between intervals. '''      
      T = np.arange(0,self.t_final+self.Dt, self.Dt)
      A = np.array([np.zeros_like(array[0]) for _ in range(np.size(T))], dtype=object)
      
      current_val = array[0]
      change_index = 0
      for i, time in enumerate(T):
        if i>0 and time>=t_array[change_index+1]:
          change_index+=1
          current_val = array[change_index]
        A[i]=current_val
      return A, T

    def run(self,t_final,Dt,entropy):
        '''Applies Gillespie algorithm up to a final time
        Output:
          P: Array with each element is the population array at a time
          T: the timesteps until final time
          EPR: Array with each element is the entropy production. Given only if entropy=='on'. '''
        self.t, self.Dt, self.t_final =0, Dt, t_final
        self.entropy = entropy  #the flag variable    
        
        self.entropy_boxes = np.zeros_like(self.pop)

        pop_array, entropy_array, t_array = [np.copy(self.pop)], [], [0]
        self.props = self.props_initialize()
        while self.t < t_final:
          if self.entropy == 'on':
            self.entropy_boxes = self.update_events()
            entropy_array.append(np.copy(self.entropy_boxes))
          else:
            self.update_events()
          pop_array.append(np.copy(self.pop)) #np.copy used so that each element is a unique copy and does not change
          
          t_array.append(self.t)
        P, T = self.physical_t(t_array,pop_array)

        if self.entropy == 'on':
          EPR, T = self.physical_t(t_array,entropy_array)
          return P.astype(float), T, EPR.astype(float)
        else:
          return P.astype(float), T
    
def average_simulations(simulations):
    """
    Compute the average and standard deviation of multiple simulation results.
    
    Parameters:
        simulations (list of arrays): List containing population arrays from multiple simulations.
        
    Returns:
        tuple: A tuple containing the mean and standard deviation arrays.
    """
    # Stack population arrays along a new axis
    stacked_populations = np.stack(simulations, axis=0)
    
    # Calculate mean and standard deviation across simulations
    mean_population = np.mean(stacked_populations, axis=0)
    std_population = np.std(stacked_populations, axis=0)
    
    return mean_population, std_population


In [ ]:
t=2
L = 1e-3
K=15
pop = np.zeros((K,K))
D = 1e-4*1e-6

h=1/K*L
d=D/h**2
threshold_distance = L

dt = 0.1
increment = 0.1

stoichs= [[0,1], [1,0]] #the chemical reactions 0->X, X->0
rates = [0.012*1e6*h,1e-3]

t_final=10
entropy = 'on'
sim = simulation(pop, stoichs, rates, d)
if entropy == 'on':
    P,T, EPR = sim.run(t_final,dt,entropy)
else:
    P,T = sim.run(t_final,dt,entropy)

plot_num_traj(P,T)
plot_num_traj(EPR,T)

plot_2d(P[-1])
plot_2d(EPR[-1])
plot_2_movies(P, T, EPR, T, increment)
